# Inference of Trained Retinanet model 
### Import Libraries

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

Using TensorFlow backend.


### Import Data and Model 

In [9]:
file_location = '../../Create-CSV-dataset/list_of_img_in_train_set_03-03.csv'
place_to_store_results = 'Evaluations/Training/'
path_img_folder = '../../../03 Data/Simple Dataset/'

# import data
file = open(file_location)
file_paths = list(file)  

print(file_paths)

# import model
model_path = os.path.join('..', 'snapshots', 'resnet50_csv_25.h5');

model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model)


labels_to_names = {0: 'Brick'};

#labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

['colorIMG_52.png\n', 'colorIMG_42.png\n', 'colorIMG_103.png\n', 'colorIMG_99.png\n', 'colorIMG_166.png\n', 'colorIMG_27.png\n', 'colorIMG_132.png\n', 'colorIMG_120.png\n', 'colorIMG_34.png\n', 'colorIMG_55.png\n', 'colorIMG_119.png\n', 'colorIMG_29.png\n', 'colorIMG_128.png\n', 'colorIMG_63.png\n', 'colorIMG_170.png\n', 'colorIMG_109.png\n', 'colorIMG_149.png\n', 'colorIMG_190.png\n', 'colorIMG_32.png\n', 'colorIMG_126.png\n', 'colorIMG_171.png\n', 'colorIMG_12.png\n', 'colorIMG_57.png\n', 'colorIMG_121.png\n', 'colorIMG_145.png\n', 'colorIMG_89.png\n', 'colorIMG_105.png\n', 'colorIMG_198.png\n', 'colorIMG_65.png\n', 'colorIMG_113.png\n', 'colorIMG_66.png\n', 'colorIMG_191.png\n', 'colorIMG_24.png\n', 'colorIMG_44.png\n', 'colorIMG_14.png\n', 'colorIMG_194.png\n', 'colorIMG_61.png\n', 'colorIMG_138.png\n', 'colorIMG_144.png\n', 'colorIMG_183.png\n', 'colorIMG_7.png\n', 'colorIMG_73.png\n', 'colorIMG_67.png\n', 'colorIMG_25.png\n', 'colorIMG_117.png\n', 'colorIMG_169.png\n', 'colorIMG_

In [4]:
def read_annotations_from_json(img_name):
    print(path_img_folder + img_name.strip('.png') + '.json')
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        annotation_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            annotation_list.append(rect)
    return annotation_list; 

def filter_bounding_boxes(bounding_boxes, scores):
    list_of_BB = []
    for box, score in zip(bounding_boxes, scores):
        if score < 0.7: 
            break
        list_of_BB.append(box)
    return list_of_BB


### Infer Images

In [10]:
accept_BB_threshold = 0.7

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    print(len(boxes[0]))
    # print detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    
    # load json file containing annotations
    annotations = read_annotations_from_json(path.strip('\n'))
    
    
    # find overlaping bounding boxes 
    predictions = filter_bounding_boxes(boxes[0], scores[0])
        #calculate intersection and union of two bounding boxes
    
    # calculate true positives, false negatives, false positives. 
    
    
    # store image
    cv2.imwrite(place_to_store_results + path.strip('\n'), draw)

# Calculate precision, recall and F1 score 
    

300
../../../03 Data/Simple Dataset/colorIMG_52.json
300
../../../03 Data/Simple Dataset/colorIMG_42.json
300
../../../03 Data/Simple Dataset/colorIMG_103.json
300
../../../03 Data/Simple Dataset/colorIMG_99.json
300
../../../03 Data/Simple Dataset/colorIMG_166.json
300
../../../03 Data/Simple Dataset/colorIMG_27.json
300
../../../03 Data/Simple Dataset/colorIMG_132.json
300
../../../03 Data/Simple Dataset/colorIMG_120.json
300
../../../03 Data/Simple Dataset/colorIMG_34.json
300
../../../03 Data/Simple Dataset/colorIMG_55.json
300
../../../03 Data/Simple Dataset/colorIMG_119.json
300
../../../03 Data/Simple Dataset/colorIMG_29.json
300
../../../03 Data/Simple Dataset/colorIMG_128.json
300
../../../03 Data/Simple Dataset/colorIMG_63.json
300
../../../03 Data/Simple Dataset/colorIMG_170.json
300
../../../03 Data/Simple Dataset/colorIMG_109.json
300
../../../03 Data/Simple Dataset/colorIMG_149.json
300
../../../03 Data/Simple Dataset/colorIMG_190.json
300
../../../03 Data/Simple Dataset/

300
../../../03 Data/Simple Dataset/colorIMG_8.json
300
../../../03 Data/Simple Dataset/colorIMG_182.json
300
../../../03 Data/Simple Dataset/colorIMG_60.json
300
../../../03 Data/Simple Dataset/colorIMG_110.json
300
../../../03 Data/Simple Dataset/colorIMG_75.json
300
../../../03 Data/Simple Dataset/colorIMG_35.json
300
../../../03 Data/Simple Dataset/colorIMG_137.json
300
../../../03 Data/Simple Dataset/colorIMG_70.json
